Problem 1: Geocode shopping centers (5 points)
The overall aim of problems 1-3 is to find out how many people live within a walking distance (1.5 km) from certain shopping centers in Helsinki.

In problem 1 aim is to find out the addresses of shopping centers and to retreive numercal coordinates for these addresses. As an output, we will have a Shapefile called shopping_centers.shp which contains the geocoded result.

Preparation: Find out the addresses for following shopping centers from the internet, and write the addresses into a text file called shopping_centers.txt:

Itis
Forum
Iso-omena
Sello
Jumbo
REDI
Tripla

Hint for "Tripla": This shopping center opened in October 10 2019, and if you are doing this exercise soon after that, the official address might not yet be in online databases. Check for an address nearby the Pasila railway station on OpenStreetMap.org and use that as input.

shopping_centers.txt should have semicolon (;) as a separator, and the file should include the following columns:

id (integer) containing an unique identifier for each shopping center
name (string) of each shopping center
addr (string) the address
See and example of how to format the text file in the lesson 3 materials. Save (and upload) the text file into your exercise repository.

Read shopping_centers.txt that you just created into a pandas DataFrame called data:

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon

#import the geocoding tool
from geopandas.tools import geocode

In [ ]:
data = pd.read_csv('shopping_centers.txt', sep= ';')
print(data.head)

In [ ]:
#verify if we are reading the data correctly
print(data.columns)  #print all column headers
print(data.iat[0,2]) #return address for item 1

Geocode the addresses using the Nominatim geocoding service. Store the output in a variable called geo:

Geocoding without a rate limiter - use for small batch of calls

In [ ]:
#geocode addresses using Nominatim api. #nominatim is rate limited to 1 call /second
geo = geocode(data['addr'], provider='nominatim', user_agent='autogis_xx',timeout=4)

#verify geocode works
geo.head()

Geocoding with a rate limiter - use when you have a larger data set Nominatim is rate limited to 1 call /second

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

#import the geocoding tool
from geopandas.tools import geocode

#Initiate geocoder Nominatim api provider
geolocator = Nominatim(user_agent = 'autogis_xx')

#create a geopy rate limiter: (to slow down the geo code requests and handle exceptions)
geocode_with_delay = RateLimiter(geolocator.geocode, min_delay_seconds=1)

#apply the geocoder with delay using the rate limiter for each row in data['addr']
data['temp'] = data['addr'].apply(geocode_with_delay)

#get point coordinates from the geopy location object on each row
data['coords'] = data['temp'].apply(lambda loc: tuple(loc.point) if loc else None)
     
#create shapely point objects to geometry column
data['geometry'] = data['coords'].apply(Point)


data.head()

In [ ]:
#NON-EDITABLE TEST CELL
# Check the geocoded output
print(geo)

In [ ]:
#NON-EDITABLE TEST CELL
# Check the data type (should be a GeoDataFrame!)
print(type(geo))

Check that the coordinate reference system of the geocoded result is correctly defined, and reproject the layer into ETRS GK-25 (EPSG:3879):

In [ ]:
#import CRS class for crs reprojection
from pyproj import CRS

# Check layer crs
print(geo.crs)

#re-project the coordinates in the geometry-column AND re-define the .crs definition
geo = geo.to_crs(CRS.from_epsg(3879))

In [ ]:
#NON-EDITABLE TEST CELL
# Check layer crs
print(geo.crs)

- Make a table join between the geocoded addresses (geo) and the original addresses (data) in order to link the numerical coordinates and the id and name of each shopping center.
- Store the output in a variable called geodata

In [ ]:
#verify the data rows are in the same order as the geo rows
print(data.head)
print(geo.head)

In [ ]:
#Join the table
geodata = geo.join(data)
print(geodata.head())

Save the output as a Shapefile called shopping_centers.shp

In [ ]:
# Define output filepath
out_fp = 'shopping_centers.shp'

# Save file
geodata.to_file(out_fp)

In [ ]:
#NON-EDITABLE TEST CELL
# Print info about output file
print("Geocoded output is stored in this file:", out_fp)

Problem 2: Create buffers around shopping centers (5 points)
- Let's continue with our case study and calculate a 1.5 km buffer around the geocoded points.

Start by creating a new column called buffer to geodata GeoDataFrame:

In [ ]:
geodata['buffer'] = None

Calculate a 1.5 km buffer for each geocoded point. Store the buffer geometry in the new buffer column.

In [ ]:
geodata['buffer'] = geodata.buffer(1500)

In [ ]:
#NON-EDITABLE TEST CELL
print(geodata.head())

In [ ]:
#NON-EDITABLE TEST CELL
# Check the data type of the first value in the buffer-column
print(type(geodata.at[0,'buffer']))

In [ ]:
#NON-EDITABLE TEST CELL
# Check the areas of your buffers in km^2
print(round(gpd.GeoSeries(geodata["buffer"]).area / 1000000))

Replace the values in geometry column with the values of buffer column:

In [ ]:
geodata.rename(columns = {'geometry':'location_point', 'buffer':'geometry'} , inplace =True)
print(geodata.head())

Problem 3: How many people live near shopping centers? (5 points)
Last step in our analysis is to make a spatial join between our buffer layer and population data in order to find out how many people live near each shopping center. We will use the same data as we did during lesson 3; a Population Grid that is available via the HSY wfs.

Alternatively, you can also download the data from the Helsinki Region Infoshare (HRI) as a shapefile (using wget).

The coordinate reference system of the population grid is ETRS GK-25 (EPSG:3879).

Steps:

- Read the population grid into a geodataframe

- Select only the useful columns from the population grid: 'asukkaita' (=population count per grid square) and 'geometry'

- Make a spatial join between your buffered point layer and population grid layer. Join the information now from buffer layer into the population grid layer

- Group the joined layer by shopping center index

- Calculate the sum of population living within 1.5 km for each shopping center.

Finally:

- Print out the population living within 1.5 km from each shopping center:

- Itis
- Forum
- Iso-omena
- Sello
- Jumbo
- REDI
- Tripla

Final print out should contain both the shopping center name and population count, for example: 26532 people live within 1.5 km from Iso-Omena.

Feel free to divide your solution into several codeblocks! Remember to comment your code :)

Read Population grid: 250m x 250m grid polygon layer that contains population information from the Helsinki Region.

- The population grid a dataset is produced by the Helsinki Region Environmental Services Authority (HSY) (see this page to access data from different years).
- You can download the data from from this link in the Helsinki Region Infroshare (HRI) open data portal.

Here, we will access the data directly from the HSY wfs:

In [ ]:
import requests
import geojson

# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format). 
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS', 
              version='2.0.0', 
              request='GetFeature', 
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018', 
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

In [ ]:
print(pop.head())

Select useful columns from the population grid: asukkaita, geometry.

In [ ]:
sub_pop = pop[['asukkaita', 'geometry']].copy()

In [ ]:
print(sub_pop.head())

Make a spatial join between your buffered point layer and population grid layer. Join the information now from buffer layer into the population grid layer

In [ ]:
#check the crs of population grids and the buffered point layers
print(sub_pop.crs)
print(geodata.crs)

In [ ]:
#define a CRS for sub_pop, since sub_pop is currently not projected to anything.
from pyproj import CRS

sub_pop.crs = CRS.from_epsg(3879).to_wkt()

print(sub_pop.crs)

In [ ]:
# Are the layers in the same projection?
geodata.crs == sub_pop.crs

In [ ]:
#join sub_pop data into geodata layer if sub_pop grid overlaps geodata, conduct a inner join.
join = gpd.sjoin(geodata,sub_pop, how="inner", op="intersects")

print(join.head())

#index_right = index of the matching polygon
#assukaita = population in the cell where the address point is located.

In [103]:
#verify if we have lost data.
print(len(join))
print(len(geodata))
print(len(sub_pop))

633
7
5832


Plot the layers on top of each other, we can observe that some of the points are located outside the populated grid squares 
- (increase figure size if you can’t see this properly!)

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with one subplot
fig, ax = plt.subplots(figsize=(15,8))

# Plot population grid
sub_pop.plot(ax=ax)

# Plot points
geodata.plot(ax=ax, color='red', markersize=5)

Visualize the joined output

Plot the points and use the assukaita column to indicate the color: 
- cmap -parameter tells to use a sequential colormap for the values (adjust color based on population density)
- markersize adjusts the size of a point, 
- scheme parameter can be used to adjust the classification method based on pysal,
- legend tells that we want to have a legend:

In [ ]:
# Create a figure with one subplot
fig, ax = plt.subplots(figsize=(10,6))

# Plot the points with population info
join.plot(ax=ax, column='asukkaita', cmap="Reds", markersize=15, scheme='quantiles', legend=True);

# Add title
plt.title("Amount of inhabitants living close the the point");

# Remove white space around the figure
plt.tight_layout()

plot the original population grid and check the overall population distribution in Helsinki:

In [ ]:
# Create a figure with one subplot
fig, ax = plt.subplots(figsize=(10,6))

# Plot the grid with population info
sub_pop.plot(ax=ax, column='asukkaita', cmap="Reds", scheme='quantiles', legend=True);

# Add title
plt.title("Population 2018 in 250 x 250 m grid squares");

# Remove white space around the figure
plt.tight_layout()

Group the joined layer by shopping center index

In [102]:
print(geodata.head)
print(join.head)

<bound method NDFrame.head of                      location_point  \
0  POINT (25504598.602 6677662.109)   
1  POINT (25496573.542 6672878.360)   
2  POINT (25485431.705 6672252.372)   
3  POINT (25489491.076 6678322.265)   
4  POINT (25497943.932 6686656.982)   
5  POINT (25498829.274 6674970.005)   
6  POINT (25496345.008 6676150.296)   

                                             address  id       name  \
0  Kauppakeskus Itis, 1-7, Itäkatu, Itäkeskus, Va...   0       Itis   
1  Salaattiasema, 14-20, Mannerheimintie, Kluuvi,...   1      Forum   
2  Sports Academy, 11, Piispansilta, Matinkylä, S...   2  Iso-Omena   
3  Lasten kappeli Arkki, 3-9, Leppävaarankatu, Sä...   3      Sello   
4  Stockmann, 3, Vantaanportinkatu, Vantaanportti...   4      Jumbo   
5  Yoga Valo, 5, Hermannin rantatie, Verkkosaari,...   5       REDI   
6  Pasilansilta, Keski-Pasila, Pasila, Keskinen s...   6     Tripla   

                                             addr  \
0            Itäkatu 1-7, 00930 Hel

In [118]:
#compute group by id to get sum of populations
groupby_join = join.groupby(by =['id']).sum()

#remove the index_right column for display
groupby_join = groupby_join[['asukkaita']]

print(groupby_join)

    asukkaita
id           
0       29199
1       78796
2       35284
3       28863
4       11103
5       41076
6       47498
